# Sicherheit in Daten: Hashing und Symmetrische VerschlüsselungIn diesem Notebook werden **zwei Hashverfahren** (SHA-2, SHA-3) und **zwei symmetrische Verschlüsselungsverfahren** (AES, Blowfish) auf einen simulierten Datensatz angewendet, um die Prinzipien von **Integrität** und **Vertraulichkeit** zu demonstrieren. Zusätzlich werden die **Integrität der Daten geprüft** sowie **Performance- und Sicherheitsaspekte** verglichen.Datensatz: *Employee Data* (Simuliert: ID, Name, Salary u.a.).

## 1. Setup und Vorbereitung

In [ ]:
import hashlibimport osimport timefrom Crypto.Cipher import AES, Blowfishfrom Crypto.Random import get_random_bytesfrom Crypto.Protocol.KDF import PBKDF2# --- Vorbereitung ---save_dir = './crypto_data_output'os.makedirs(save_dir, exist_ok=True)file_name = 'employee_data.csv'file_path = os.path.join(save_dir, file_name)# Erzeuge eine simulierte Datendateidata_content = "ID,Name,Salary,Department\n"for i in range(1, 10000):    data_content += f"{i},Employee_{i},{50000 + i % 10 * 1000},R&D\n"with open(file_path, 'w') as f:    f.write(data_content)print(f"Simulierte Datei erstellt: {file_path} ({os.path.getsize(file_path)/1024:.2f} KB)")

## 2. Hashing (Integritätsprüfung und Authentizität)Hashing erzeugt einen **eindeutigen Fingerabdruck** der Daten. Wir verwenden eine Hilfsfunktion, die den Hash blockweise berechnet, um auch größere Dateien effizient zu verarbeiten.

In [ ]:
# Hilfsfunktion zur blockweisen Hash-Berechnungdef calculate_file_hash(file_path, algorithm='sha256'):    hasher = hashlib.new(algorithm)    start_time = time.time()    with open(file_path, 'rb') as f:        while chunk := f.read(4096):            hasher.update(chunk)    end_time = time.time()    return hasher.hexdigest(), (end_time - start_time) * 1000 # Zeit in Millisekunden# --- Hashing der Originaldatei ---print("### Hashing der Originaldatei ###")# SHA-256sha256_hash, sha256_time = calculate_file_hash(file_path, 'sha256')print(f"SHA-256 Hash: {sha256_hash[:20]}... (Dauer: {sha256_time:.4f} ms)")REF_HASH_SHA256 = sha256_hash # Speichere Hash als Referenz (SOLL-Zustand)# SHA-3 (mit 256 Bit Länge)sha3_hash, sha3_time = calculate_file_hash(file_path, 'sha3_256')print(f"SHA-3 Hash:   {sha3_hash[:20]}... (Dauer: {sha3_time:.4f} ms)")REF_HASH_SHA3 = sha3_hash # Speichere Hash als Referenz (SOLL-Zustand)

### Integritätsprüfung (Simuliert)Um die Integrität zu prüfen, wird der Hash der Datei erneut berechnet und mit dem **Referenz-Hash (SOLL-Zustand)** verglichen.

In [ ]:
# --- Simulation einer Manipulation (Optional: Auskommentieren, um die Prüfung zu testen) ---# with open(file_path, 'a') as f:#     f.write('\nMANIPULIERT') # print("\n*** ACHTUNG: MANIPULATION DER DATEI SIMULIERT ***\n")print("### Integritätsprüfung nach SHA-256 ###")check_hash, _ = calculate_file_hash(file_path, 'sha256')if check_hash == REF_HASH_SHA256:    print("✅ Integrität OK! (SHA-256)")else:    print(f"❌ Integrität verletzt! (SHA-256) Ist-Hash: {check_hash[:20]}...\n")    print("### Integritätsprüfung nach SHA-3 ###")check_hash, _ = calculate_file_hash(file_path, 'sha3_256')if check_hash == REF_HASH_SHA3:    print("✅ Integrität OK! (SHA-3)")else:    print(f"❌ Integrität verletzt! (SHA-3) Ist-Hash: {check_hash[:20]}...\n")

## 3. Symmetrische Verschlüsselung (Vertraulichkeit)Die symmetrische Verschlüsselung nutzt den gleichen Schlüssel zum Ver- und Entschlüsseln, um die **Vertraulichkeit** der Daten zu sichern. **Hinweis:** Symmetrische Verschlüsselung sichert **nicht** die Integrität; dies müsste durch Hashing oder einen Message Authentication Code (MAC) ergänzt werden.

### AES (Advanced Encryption Standard)

In [ ]:
PASSWORD = "EinSehrSicheresPasswort123"SALT = get_random_bytes(16) # Einzigartiger Salt pro Verschlüsselung!key = PBKDF2(PASSWORD, SALT, dkLen=32, count=100000) # 256 Bit Schlüssel ableiteniv = get_random_bytes(16) # Initialisierungsvektorwith open(file_path, 'rb') as f:    plaintext = f.read()# Fülle Daten auf Blockgröße auf (Padding)BS = AES.block_sizepadding_length = BS - (len(plaintext) % BS)padded_plaintext = plaintext + bytes([padding_length]) * padding_lengthstart_time = time.time()cipher = AES.new(key, AES.MODE_CBC, iv=iv)ciphertext_aes = cipher.encrypt(padded_plaintext)aes_time = (time.time() - start_time) * 1000# Verschl. Datei speichern (Hier müsste zusätzlich der Hash/MAC für die Integrität gespeichert werden!)enc_file_path_aes = os.path.join(save_dir, 'employee_data_aes_encrypted.bin')with open(enc_file_path_aes, 'wb') as f:    f.write(SALT + iv + ciphertext_aes)print(f"AES-Verschlüsselung abgeschlossen. Dauer: {aes_time:.4f} ms")# Beispiel für Entschlüsselung (Zur Funktionsprüfung)decipher = AES.new(key, AES.MODE_CBC, iv=iv)decrypted = decipher.decrypt(ciphertext_aes)decrypted = decrypted[:-decrypted[-1]] # Padding entfernenprint("AES Entschlüsselung erfolgreich.")

### Blowfish

In [ ]:
# Blowfish verwendet ähnliche Schlüsselableitung und PaddingPASSWORD = "EinSehrSicheresPasswort123"SALT_BF = get_random_bytes(16)key_bf = PBKDF2(PASSWORD, SALT_BF, dkLen=32, count=100000) # Blowfish unterstützt Schlüssel bis 448 Bitiv_bf = get_random_bytes(8) # Blowfish hat eine Blockgröße von 8 Byteswith open(file_path, 'rb') as f:    plaintext = f.read()# Fülle Daten auf Blockgröße von Blowfish (8 Bytes) aufBS_BF = 8padding_length_bf = BS_BF - (len(plaintext) % BS_BF)padded_plaintext_bf = plaintext + bytes([padding_length_bf]) * padding_length_bfstart_time = time.time()cipher_bf = Blowfish.new(key_bf, Blowfish.MODE_CBC, iv=iv_bf)ciphertext_bf = cipher_bf.encrypt(padded_plaintext_bf)blowfish_time = (time.time() - start_time) * 1000# Verschl. Datei speichernenc_file_path_bf = os.path.join(save_dir, 'employee_data_blowfish_encrypted.bin')with open(enc_file_path_bf, 'wb') as f:    f.write(SALT_BF + iv_bf + ciphertext_bf)print(f"Blowfish-Verschlüsselung abgeschlossen. Dauer: {blowfish_time:.4f} ms")# Beispiel für Entschlüsselung (Zur Funktionsprüfung)decipher_bf = Blowfish.new(key_bf, Blowfish.MODE_CBC, iv=iv_bf)decrypted_bf = decipher_bf.decrypt(ciphertext_bf)decrypted_bf = decrypted_bf[:-decrypted_bf[-1]] # Padding entfernenprint("Blowfish Entschlüsselung erfolgreich.")

## 4. Performance-, Sicherheits- und Einsatz-VergleichHier fassen wir die Ergebnisse zusammen und bewerten die vier Verfahren hinsichtlich ihrer Schlüsselkriterien.

In [ ]:
import pandas as pdprint("### Performanz-Vergleich (Verschlüsselung/Hashing) ###")performance_data = {    'Verfahren': ['SHA-256', 'SHA-3', 'AES-256 (CBC)', 'Blowfish (CBC)'],    'Funktion': ['Hashing', 'Hashing', 'Verschlüsselung', 'Verschlüsselung'],    'Dauer (ms)': [sha256_time, sha3_time, aes_time, blowfish_time],}df_perf = pd.DataFrame(performance_data).sort_values(by='Dauer (ms)')print(df_perf.to_markdown(index=False, floatfmt=".4f"))

### Sicherheits- und Einsatz-Analyse| Verfahren | Sicherheitsziel | Sicherheit heute | Performance-Faktor | Typischer Einsatz || :--- | :--- | :--- | :--- | :--- || **SHA-256** | **Integrität, Authentizität** | Hoch. Standardverfahren. | Sehr schnell, oft Hardware-optimiert. | Passwort-Hashing, Digital Signaturen, Blockchain. || **SHA-3** | **Integrität, Authentizität** | Hoch. Modernste Standards. | Schnell, oft etwas langsamer als SHA-2 (Software). | Alternative zu SHA-2, z.B. bei Bedenken gegen NSA-Design. || **AES** | **Vertraulichkeit** | Sehr hoch. Weltweiter Standard. | Sehr schnell, meist **Hardware-beschleunigt (AES-NI)**. | TLS/SSL (HTTPS), Festplattenverschlüsselung (BitLocker), VPN. || **Blowfish** | **Vertraulichkeit** | Akzeptabel, aber veraltet. | Langsamer als AES, keine Hardware-Beschleunigung. | Ältere Anwendungen, Passwort-Speicherung (mit Anpassung). **AES wird bevorzugt.** |#### FazitDie Kombination aus **AES (Vertraulichkeit)** und **SHA-256 (Integrität)** ist der Goldstandard für moderne Sicherheitssysteme, da sie schnell, sicher und weit verbreitet ist. Für eine echte Integritäts- und Authentizitätssicherung in verschlüsselten Daten sollte idealerweise ein **Authenticated Encryption Mode** (z.B. **AES-GCM**) oder ein **MAC** (Message Authentication Code) anstelle eines einfachen Hashes verwendet werden.